# 熟悉Chisel语言

## 引入Chisel的包
在Scala中开发Chisel需要引入Chisel的一些包，source/load-ivy.sc中指明了需要使用的Chisel版本

`import`引入了需要使用的Chisel包

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

In [6]:
0.S
3.U(2.W)

res5_0: SInt = SInt<1>(0)
res5_1: UInt = UInt<2>(3)

Chisel有带符号数(SInt)和不带符号数(UInt)
- `0.S`定义了一个Chisel带符号数，其值为0
- `3.U(2.W)`定义了一个Chisel无符号数，其值是3，宽度是2，其中`2.W`是Chisel的宽度类型width type，用来指定宽度

## 1 选择器Mux

In [14]:
class MyOperatorsTwo extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(4.W))
    val out_mux = Output(UInt(4.W))
    val out_cat = Output(UInt(4.W))
  })

  val s = true.B
  io.out_mux := Mux(s, 3.U, 0.U) // should return 3.U, since s is true
  io.out_cat := Cat(2.U, 1.U)    // concatenates 2 (b10) with 1 (b1) to give 5 (101)
}

println(getVerilog(new MyOperatorsTwo)) 

test(new MyOperatorsTwo) { c =>
  c.io.out_mux.expect(3.U)
  c.io.out_cat.expect(5.U)
}

Elaborating design...
Done elaborating.
module MyOperatorsTwo(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out_mux,
  output [3:0] io_out_cat
);
  assign io_out_mux = 4'h3; // @[cmd13.sc 9:20]
  assign io_out_cat = 4'h5; // @[Cat.scala 30:58]
endmodule

Elaborating design...
Done elaborating.
test MyOperatorsTwo Success: 0 tests passed in 2 cycles in 0.002086 seconds 958.77 Hz


defined class MyOperatorsTwo

## 2 寄存器Register
1. Reg()
2. RegInit()
3. RegNext()

In [25]:
class MyRegister extends Module{
    val io=IO(new Bundle{
        val d=Input(UInt(8.W))
        val q=Output(UInt(8.W))
    })

    // val regInstance=Reg(UInt(8.W)) // 创建一个8bits的寄存器

    // val regInstance=RegInit(0.U(8.W)) // 创建一个8bits的寄存器，初始化为0

    val regInstance=RegNext(io.d) // 创建一个寄存器，并且寄存器的输入连接到d

    // 连接寄存器的输入输出
    // regInstance:=io.d
    io.q:=regInstance
}

// println(getVerilog(new MyRegister))

// 测试MyRegister
test(new MyRegister){c =>
    c.io.d.poke(1.U)
    var q=c.io.q.peek()
    println(q)
    c.clock.step(1) // 延迟一个周期
    q=c.io.q.peek()
    println(q)
    c.io.q.expect(1.U)    

    c.clock.step(1)
    c.io.d.poke(3.U)
    q=c.io.q.peek()
    println(q)
    c.clock.step(1) // 延迟一个周期
    q=c.io.q.peek()
    println(q)
}

Elaborating design...
Done elaborating.
UInt<8>(0)
UInt<8>(1)
UInt<8>(1)
UInt<8>(3)
test MyRegister Success: 0 tests passed in 5 cycles in 0.004269 seconds 1171.29 Hz


defined class MyRegister

## 3 计数器Counter
> 需要使用到`Mux`和`Register`

In [36]:
// 一个从0-9的计数器
class MyCounter extends Module{
    val io=IO(new Bundle{
        val count=Output(UInt(4.W))
    })

    val register=RegInit(0.U(4.W))
    val mux=Mux(register===9.U, 0.U, register+1.U)

    register:=mux
    io.count:=register
}

test(new MyCounter){c=>
    var count=c.io.count.peek()
    println(count)

    c.clock.step(1)
    count=c.io.count.peek()
    println(count)

    c.clock.step(1)
    count=c.io.count.peek()
    println(count)   

    c.clock.step(6)
    count=c.io.count.peek()
    println(count)  

    c.clock.step(1)
    count=c.io.count.peek()
    println(count) 
}

Elaborating design...
Done elaborating.
UInt<4>(0)
UInt<4>(1)
UInt<4>(2)
UInt<4>(8)
UInt<4>(9)
test MyCounter Success: 0 tests passed in 11 cycles in 0.008512 seconds 1292.35 Hz


defined class MyCounter

## Buddle & Vec
> Buddle和C里面的Struct类似; Vec和C里面的Array相似

Vec有如下三个作用：
1. mux
2. register file
3. parametrization

# Build & Test
1. 编写Chisel代码
2. 获得Verilog代码
3. Debug代码

> Scala的<u>构建(build)</u>工具有sbt和Mill，sbt可以构建Scala项目，调试Scala项目，并且还会下载项目所需要的Chisel库及Scala版本